In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.9 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.6 MB/s 


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 9.6 MB/s 
     |████████████████████████████████| 212 kB 55.4 MB/s 
     |████████████████████████████████| 115 kB 60.5 MB/s 
     |████████████████████████████████| 127 kB 56.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install speechbrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 496 kB 10.8 MB/s 
     |████████████████████████████████| 1.3 MB 53.5 MB/s 
     |████████████████████████████████| 750.6 MB 11 kB/s 
     |████████████████████████████████| 109 kB 56.1 MB/s 
     |████████████████████████████████| 546 kB 61.8 MB/s 
     |████████████████████████████████| 3.7 MB 48.0 MB/s 
     |████████████████████████████████| 3.7 MB 59.4 MB/s 
     |████████████████████████████████| 2.9 MB 41.5 MB/s 
  Created wheel for hyperpyyaml: filename=HyperPyYAML-1.0.1-py3-none-any.whl size=15192 sha256=98de6040e0425806f847fb2aca45f6aa4bf2b82b1ca0cce8c43cb9472da9ca14
  Stored in directory: /root/.cache/pip/wheels/6c/87/65/266d722c3932f81f16332ce842e972be8421e3a9cd3771766b
Successfully built hyperpyyaml
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully u

# Importando bibliotecas

In [4]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile

In [5]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [6]:
from datasets import load_dataset, load_metric

# Puxa dataset

In [7]:
from datasets import load_dataset, Audio

In [8]:
import pandas as pd

In [9]:
import numpy as np


# Rodando Classificador na Original

In [10]:
import pickle
label2id = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_label2id.pkl", "rb"))
id2label = pickle.load(open("/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_id2label.pkl", "rb"))

In [11]:
import pandas as pd
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_original.pkl')

In [12]:
test = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_test.pkl')

In [13]:
val = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_validation.pkl')

In [14]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
from speechbrain.processing.speech_augmentation import SpeedPerturb

perturbator = SpeedPerturb(orig_freq=8000, speeds=[95 + i for i in range(11)], perturb_prob=1.0)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
import torch
import numpy as np
def speed_aug(pert, audio_dict, n_aug):
  resp = audio_dict.copy()
  audio = torch.from_numpy(resp['array'].astype(np.float32)).unsqueeze(0)

  res = []

  for i in range(n_aug):
    perturbed = perturbator(audio).squeeze().numpy()
    resp['array'] = np.array(perturbed).astype(np.float64)
    res.append(resp)
  torch.cuda.empty_cache()
  return res




In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()
minds_aug = train.copy()

minds_aug['audio_aug'] = minds_aug['audio'].progress_apply(lambda x:speed_aug(perturbator, x, 10))

  0%|          | 0/5110 [00:00<?, ?it/s]

In [ ]:
minds_aug= minds_aug.explode('audio_aug').reset_index(drop=True)
minds_aug.shape

(51100, 4)

In [ ]:
minds_aug.head()

,file,audio,label,audio_aug
0,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
1,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
2,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
3,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...
4,/root/.cache/huggingface/datasets/downloads/ex...,{'path': '/root/.cache/huggingface/datasets/do...,2,{'path': '/root/.cache/huggingface/datasets/do...


In [ ]:
minds_aug_fim = minds_aug.drop('audio', axis =1)
minds_aug_fim.shape

(51100, 3)

In [ ]:
minds_aug_fim.rename(columns={'audio_aug':'audio'},inplace=True)

train = train.append([minds_aug_fim])
train.shape

(56210, 3)

In [ ]:
train.to_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_speed_pertub.pkl')

In [15]:
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/audio_ml/datasets/superb_ks_speed_pertub.pkl')
train = train.sample(frac=1).copy()

In [16]:
train.shape

(56210, 3)

In [17]:
def convert_np(audio_dict):
  resp = audio_dict.copy()
  resp['array'] = np.array(resp['array']).astype(np.float32)
  return resp

In [18]:
train['audio'] = train['audio'].apply(lambda x:convert_np(x))

In [19]:
from datasets import Dataset
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
val = Dataset.from_pandas(val)

In [20]:
max_duration = 1.0
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [21]:
encoded_train = train.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_test = test.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

encoded_val = val.map(preprocess_function, remove_columns=["audio", "file"], batched=True)

  0%|          | 0/57 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [28]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

loading configuration file https://huggingface.co/facebook/wav2vec2-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c7746642f045322fd01afa31271dd490e677ea11999e68660a92619ec7c892b4.ce1f96bfaf3d7475cb8187b9668c7f19437ade45fb9ceb78d2b06a2cec198015
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,

In [29]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    warmup_ratio=0.1,
    logging_steps=1,
    num_train_epochs=3,
    metric_for_best_model="accuracy",
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [30]:
def compute_metrics(eval_pred):
    acc = load_metric("accuracy")
    f1 = load_metric("f1", average='macro')
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)



In [32]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 56210
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 1317


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.738600,0.675346,0.849367,0.602647
1,0.346300,0.255372,0.963960,0.937753
2,0.228300,0.202608,0.971021,0.951549


***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32


***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32
***** Running Evaluation *****
  Num examples = 6798
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1317, training_loss=0.6712905318563239, metrics={'train_runtime': 3070.2911, 'train_samples_per_second': 54.923, 'train_steps_per_second': 0.429, 'total_flos': 1.53080800664832e+18, 'train_loss': 0.6712905318563239, 'epoch': 3.0})

In [33]:
predictions = trainer.predict(encoded_test)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 3081
  Batch size = 32


(3081, 12) (3081,)


In [34]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [35]:
acc = load_metric("accuracy")
f1 = load_metric("f1", average='macro')

In [36]:
accuracy = acc.compute(predictions=preds, references=predictions.label_ids)["accuracy"]
accuracy

0.8682246024018176

In [37]:
f1 = f1.compute(predictions=preds, references=predictions.label_ids, average="macro")["f1"]
f1

0.8439392429535458